# Tutorial MongoDB CRUD and Aggregation, <div style="color:red">Module Assigment 1</div>

----
student's name: Darshan Morkar

student's email: d.morkar@oth-aw.de

----

<br><br>


<div style="color:red">Attention, this tutorial is an Module Assigment, it is used for grade. I refere to the module course where you should hand in this notebook with your code, description and explanations</div>


<br><br>



Download the mongodb server and install it on your machine.
the notebook requires `pymongo`: `pip install pymongo`  this as well.


Mac command for running the mongoDB Server after installing the server using brew: `brew services start mongodb/brew/mongodb-community`


Make yourself familiar with the possibilities to access and wrangle information using pymongo and the underlying query mechanism of mongoDB. A document describing the aggregation pipeline of mongoDB is located in the moodle course.




In [126]:
pip install pymongo

Note: you may need to restart the kernel to use updated packages.


In [127]:
from pymongo import MongoClient

client = MongoClient("mongodb://localhost:27017/")
db = client.shopDB
print(db.list_collection_names())

[]


#### Connection establishment

Python based Connection Management, URL of server is necessary, typically `localhost`

Database name: _`shopDB`_

Collections are referenced using the name of the collections.

Collection is created if not already existant in DB


In [128]:
from pymongo import MongoClient
from bson import ObjectId
from datetime import datetime

client = MongoClient("mongodb://localhost:27017")
db = client["shopDB"]

users = db["users"]
products = db["products"]
orders = db["orders"]
reviews = db["reviews"]


# Reset
users.delete_many({})
products.delete_many({})
orders.delete_many({})
reviews.delete_many({})

DeleteResult({'n': 0, 'ok': 1.0}, acknowledged=True)

#### Database content 
some entries are added (create / inserted) to the corresponding collection

In [129]:

# Insert users
users.insert_many([
    {
        "_id": ObjectId("65f100000000000000000001"),
        "name": "Alice Müller",
        "email": "alice@example.com",
        "createdAt": datetime(2025, 1, 10, 9, 0, 0),
        "address": {"street": "Main Street 1", "city": "Berlin", "zip": "10115", "country": "DE"},
        "tags": ["premium", "newsletter"],
    },
    {
        "_id": ObjectId("65f100000000000000000002"),
        "name": "Bob Schmidt",
        "email": "bob@example.com",
        "createdAt": datetime(2025, 1, 12, 15, 30, 0),
        "address": {"street": "Oak Avenue 5", "city": "Hamburg", "zip": "20095", "country": "DE"},
        "tags": ["standard"],
    },
])

# Insert products
products.insert_many([
    {"_id": ObjectId("65f200000000000000000001"), "name": "Mechanical Keyboard",
     "category": "electronics", "price": 129.99, "stock": 42,
     "tags": ["keyboard", "mechanical", "gaming"], "createdAt": datetime(2025,1,8,10)},
    {"_id": ObjectId("65f200000000000000000002"), "name": "Noise-Cancelling Headphones",
     "category": "electronics", "price": 199.00, "stock": 15,
     "tags": ["audio","headphones"], "createdAt": datetime(2025,1,9,11,30)},
    {"_id": ObjectId("65f200000000000000000003"), "name": "Ergonomic Office Chair",
     "category": "furniture", "price": 299.00, "stock": 7,
     "tags": ["office","chair"], "createdAt": datetime(2025,1,11,13)},
])

# Insert orders
orders.insert_many([
    {
        "_id": ObjectId("65f300000000000000000001"),
        "userId": ObjectId("65f100000000000000000001"),
        "createdAt": datetime(2025,1,15,10,15),
        "status": "shipped",
        "items": [
            {"productId": ObjectId("65f200000000000000000001"), "nameSnapshot": "Mechanical Keyboard", "quantity": 1, "priceAtPurchase": 119.99},
            {"productId": ObjectId("65f200000000000000000002"), "nameSnapshot": "Noise-Cancelling Headphones", "quantity": 1, "priceAtPurchase": 189.00},
        ],
        "shippingAddress": {"street":"Main Street 1","city":"Berlin","zip":"10115","country":"DE"},
        "payment": {"method":"credit_card","paid":True,"totalAmount":308.99},
    },
    {
        "_id": ObjectId("65f300000000000000000002"),
        "userId": ObjectId("65f100000000000000000002"),
        "createdAt": datetime(2025,1,16,16,45),
        "status": "processing",
        "items": [
            {"productId": ObjectId("65f200000000000000000003"), "nameSnapshot": "Ergonomic Office Chair", "quantity": 1, 
             "priceAtPurchase": 289.00},
        ],
        "shippingAddress": {"street":"Oak Avenue 5","city":"Hamburg","zip":"20095","country":"DE"},
        "payment": {"method":"paypal","paid":False,"totalAmount":289.00},
    },
])

# Insert reviews
reviews.insert_many([
    {"_id": ObjectId("65f400000000000000000001"),
     "productId": ObjectId("65f200000000000000000001"),
     "userId": ObjectId("65f100000000000000000001"),
     "rating": 5, "title": "Great keyboard!",
     "text": "Feels amazing to type, perfect for coding.",
     "createdAt": datetime(2025,1,17,9)},
    {"_id": ObjectId("65f400000000000000000002"),
     "productId": ObjectId("65f200000000000000000002"),
     "userId": ObjectId("65f100000000000000000002"),
     "rating": 4, "title": "Good sound",
     "text": "Noise cancelling is solid, but a bit tight.",
     "createdAt": datetime(2025,1,18,14,30)},
]

)

print("Data inserted successfully")

Data inserted successfully


Adding individual Entries are also possible.

In [130]:

# 3.1 CREATE: a User
new_user = {
    "name": "Charlie Weber",
    "email": "charlie@example.com",
    "createdAt": datetime.utcnow(),
    "address": {
        "street": "Example St 10",
        "city": "Munich",
        "zip": "80331",
        "country": "DE",
    },
    "tags": ["standard"],
}
insert_result = users.insert_one(new_user)
print("Inserted new user with _id:", insert_result.inserted_id)

Inserted new user with _id: 697d09d44a34610543cd41c1


/var/folders/4h/lyjhdgz544d83sxbs288fh7m0000gn/T/ipykernel_13833/1784188016.py:5: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  "createdAt": datetime.utcnow(),


In [131]:
# 3.2 READ: products of categoryelectronics, price < 150
print("\nElectronics products with price < 150:")
for doc in products.find({"category": "electronics", "price": {"$lt": 150}}):
    print(doc)

# READ: Alice's Orders  
alice = users.find_one({"email": "alice@example.com"})
print("\nOrders for Alice:")
for order in orders.find({"userId": alice["_id"]}):
    print(order)

# READ: get reviews for specific product
keyboard = products.find_one({"name": "Mechanical Keyboard"})
print("\nReviews for Mechanical Keyboard:")
for review in reviews.find({"productId": keyboard["_id"]}):
    print(review)

# 3.3 UPDATE: change order status
print("\nUpdating order status for Bob's order...")
orders.update_one(
    {"_id": ObjectId("65f300000000000000000002")},
    {"$set": {"status": "shipped", "payment.paid": True}},
)
print("Updated order:\n", orders.find_one({"_id": ObjectId("65f300000000000000000002")}))

# UPDATE: Increase all elements by 10 money ;)
print("\nIncreasing price of all electronics products by 10...")
products.update_many(
    {"category": "electronics"},
    {"$inc": {"price": 10}},
)
for doc in products.find({"category": "electronics"}):
    print(doc)

# 3.4 DELETE
print("\nDeleting reviews with rating <= 2 (if any)...")
delete_result = reviews.delete_many({"rating": {"$lte": 2}})
print("Deleted count:", delete_result.deleted_count)


Electronics products with price < 150:
{'_id': ObjectId('65f200000000000000000001'), 'name': 'Mechanical Keyboard', 'category': 'electronics', 'price': 129.99, 'stock': 42, 'tags': ['keyboard', 'mechanical', 'gaming'], 'createdAt': datetime.datetime(2025, 1, 8, 10, 0)}

Orders for Alice:
{'_id': ObjectId('65f300000000000000000001'), 'userId': ObjectId('65f100000000000000000001'), 'createdAt': datetime.datetime(2025, 1, 15, 10, 15), 'status': 'shipped', 'items': [{'productId': ObjectId('65f200000000000000000001'), 'nameSnapshot': 'Mechanical Keyboard', 'quantity': 1, 'priceAtPurchase': 119.99}, {'productId': ObjectId('65f200000000000000000002'), 'nameSnapshot': 'Noise-Cancelling Headphones', 'quantity': 1, 'priceAtPurchase': 189.0}], 'shippingAddress': {'street': 'Main Street 1', 'city': 'Berlin', 'zip': '10115', 'country': 'DE'}, 'payment': {'method': 'credit_card', 'paid': True, 'totalAmount': 308.99}}

Reviews for Mechanical Keyboard:
{'_id': ObjectId('65f400000000000000000001'), 'p

# Task (MongoDB,  Nested Documents)

<div style="color:orange">Create for each task a specific python function (<code>taks1(...), task2(...), task3(...)</code>)!</div> 

<br>

<div style="color:red">Make also sure to compare your solution against the solution provided by chatgpt. Discuss overlaps or divergences of the solutions.</div>

In addition to the implementation tasks, discuss where atomicity and multi-document transactions are required in this scenario (user creation, stock validation and update, order insertion).
Briefly explain the potential consequences if no atomicity / transaction support is used (e.g. race conditions, overselling, inconsistent orders).
Finally, provide a concrete solution that uses MongoDB transactions (via pymongo and sessions) to ensure that user handling, stock updates, and order creation are executed atomically.

<br>

**Be aware to combine all subTask Functions in the last function**

Implement python code to create a **new order in MongoDB** with the following requirements:


1. **User handling (embedded user document in the order)**  <code>taks1(...)</code>
   - The order is placed by a user identified by some unique field (e.g. `email`).  
   - Check in the MongoDB `users` collection whether this user already exists.  
   - If the user does **not** exist, insert a **new user document** into the `users` collection. Align the format of the new document with the previously stored documents.
   - In the order document, embed a **nested `user` subdocument** (e.g. containing `userId`, `name`, `email`, etc.), so that user information is directly available inside the order.


In [132]:
from datetime import datetime
from bson import ObjectId

def task1(email, name, address, users, session):
    """
    Checks if a user exists. If not, creates one.
    Returns an embedded user snapshot.
    """

    user = users.find_one({"email": email}, session=session)

    if not user:
        user = {
            "name": name,
            "email": email,
            "createdAt": datetime.utcnow(),
            "address": address,
            "tags": ["standard"]
        }
        result = users.insert_one(user, session=session)
        user["_id"] = result.inserted_id

    embedded_user = {
        "userId": user["_id"],
        "name": user["name"],
        "email": user["email"],
        "address": {
            "city": user["address"]["city"],
            "country": user["address"]["country"]
        }
    }

    return embedded_user



2. **Order content with embedded product descriptions**  <code>taks2(...)</code>
   - The order must contain **at least one item**.
   - Each item refers to a product stored in the `products` collection (e.g. via `productId`).  
   - For each item, fetch the corresponding product from `products` and embed a **nested product description** inside the order, for example:
     ```json
     {
       "productId": "...",
       "name": "...",
       "priceAtPurchase": ...,
       "quantity": ...
     }
     ```
   - This way, the order stores a **snapshot of the product data** at the time of purchase.


In [133]:
def task2(order_items, products, session):
    """
    Creates product snapshots for the order.
    """

    if not order_items:
        raise ValueError("Order must contain at least one item")

    embedded_items = []

    for item in order_items:
        product = products.find_one({"_id": item["productId"]}, session=session)

        if not product:
            raise ValueError("Product not found")

        embedded_items.append({
            "productId": product["_id"],
            "name": product["name"],
            "priceAtPurchase": product["price"],
            "quantity": item["quantity"]
        })

    return embedded_items



3. **Stock validation and update in MongoDB**  
   - Before inserting the order into the `orders` collection:
     - For each item, query the `products` collection and check whether there is **sufficient `stock`** available.  
     - If **any** product does not have enough stock, **abort** the operation (do not insert the order, do not modify stock).  
     - If all products have sufficient stock, **decrease the stock** in `products` for each item (e.g. using `$inc: { stock: -quantity }`).




In [134]:
def task3(order_items, products, session):
    """
    Validates stock and updates inventory atomically.
    """

    # Check stock first
    for item in order_items:
        product = products.find_one({"_id": item["productId"]}, session=session)

        if not product:
            raise ValueError("Product not found")

        if product["stock"] < item["quantity"]:
            raise ValueError("Insufficient stock")

    # Update stock only if all checks pass
    for item in order_items:
        products.update_one(
            {"_id": item["productId"]},
            {"$inc": {"stock": -item["quantity"]}},
            session=session
        )

    return True


4. **Order creation in the `orders` collection**  
   - Only after:
     - (a) the user has been ensured to exist (and created if necessary), and  
     - (b) all stock levels have been validated and updated,  
   - insert a new document into the `orders` collection.  
   - The order document should contain:
     - an `_id` or `orderId`,  
     - a `createdAt` timestamp,  
     - the embedded `user` subdocument,  
     - an `items` array containing the embedded product descriptions with quantities,  
     - optional payment/total fields (e.g. `totalAmount`).


In [135]:
def task4(embedded_user, embedded_items, orders, session):
    """
    Creates and inserts the final order document.
    """

    if not embedded_items:
        raise ValueError("Order must contain at least one item")

    total_amount = sum(
        item["priceAtPurchase"] * item["quantity"]
        for item in embedded_items
    )

    order_doc = {
        "_id": ObjectId(),
        "createdAt": datetime.utcnow(),
        "user": embedded_user,
        "items": embedded_items,
        "payment": {
            "method": "unknown",
            "paid": False,
            "totalAmount": total_amount
        }
    }

    orders.insert_one(order_doc, session=session)

    return order_doc


In [136]:
from pymongo import MongoClient

client = MongoClient("mongodb://localhost:27017")
db = client["shopOrders"]

def create_order_atomic(email, name, address, order_items):
    """
    Creates a full order atomically using MongoDB transactions.
    """

    with client.start_session() as session:
        with session.start_transaction():

            # Task 1 — User handling
            embedded_user = task1(email, name, address, db.users, session)

            # Task 2 — Product snapshots
            embedded_items = task2(order_items, db.products, session)

            # Task 3 — Stock validation and update
            task3(order_items, db.products, session)

            # Task 4 — Order creation
            order_doc = task4(embedded_user, embedded_items, db.orders, session)

            return order_doc


**Atomicity is required because order creation consists of multiple dependent database operations. If one operation fails in between, the database could become inconsistent. Therefore, these operations should behave as a single unit. If atomicity or transaction support is not used, multiple problems can occur because order creation involves several dependent database operations. One major issue is race conditions, where two customers try to buy the last item at the same time, causing the system to oversell products. Another problem is inconsistent data, for example if stock is reduced but the order document fails to be inserted, the inventory becomes incorrect. Similarly, a user might be created but the order fails, leaving unused or duplicate user records. Without transactions, the database can end up in a partially updated and unreliable state, which can affect business logic and reporting.**

## Aggregations in MongoDB

We now want to apply specific aggregation operations to retrieve combined information from useser and their orders. The json file contains a larger dataset which has to be analyzed.

In [137]:
import json

with open("orders.json", encoding="utf-8") as f:
    data = json.load(f)

db.users.insert_many(data["users"])
db.products.insert_many(data["products"])
db.orders.insert_many(data["orders"])
db.reviews.insert_many(data["reviews"])

InsertManyResult(['65f400000000000000000001', '65f400000000000000000002', '65f400000000000000000003', '65f400000000000000000004', '65f400000000000000000005', '65f400000000000000000006', '65f400000000000000000007', '65f400000000000000000008', '65f400000000000000000009', '65f400000000000000000010', '65f400000000000000000011', '65f400000000000000000012', '65f400000000000000000013', '65f400000000000000000014', '65f400000000000000000015', '65f400000000000000000016', '65f400000000000000000017', '65f400000000000000000018', '65f400000000000000000019', '65f400000000000000000020', '65f400000000000000000021', '65f400000000000000000022', '65f400000000000000000023', '65f400000000000000000024', '65f400000000000000000025', '65f400000000000000000026', '65f400000000000000000027', '65f400000000000000000028', '65f400000000000000000029', '65f400000000000000000030', '65f400000000000000000031', '65f400000000000000000032', '65f400000000000000000033', '65f400000000000000000034', '65f400000000000000000035', '6

Example pipeline for getting mean information for order values:

In [138]:
pipeline = [
    {
        "$group": {
            "_id": None,
            "avgOrderValue": {"$avg": "$payment.totalAmount"},
            "minOrderValue": {"$min": "$payment.totalAmount"},
            "maxOrderValue": {"$max": "$payment.totalAmount"},
            "countOrders": {"$sum": 1},
        }
    }
]

list(db.orders.aggregate(pipeline))

[{'_id': None,
  'avgOrderValue': 868.71068,
  'minOrderValue': 11.53,
  'maxOrderValue': 3691.32,
  'countOrders': 250}]

### Get all Users, who ordered more than twice of the avarage price of an order

In [139]:
pipeline = [
    # 1: Durchschnitt berechnen
    {
        "$group": {
            "_id": None,
            "avgOrderValue": { "$avg": "$payment.totalAmount" }
        }
    },

    # 2: Den Durchschnittswert in die nächste Pipelinephase übertragen
    {
        "$lookup": {
            "from": "orders",
            "pipeline": [
                {
                    "$match": {
                        # Filter: Bestellungen > 2 × Durchschnitt
                        "$expr": {
                            "$gt": [
                                "$payment.totalAmount",
                                { "$multiply": [ "$$avgValue", 2 ] }
                            ]
                        }
                    }
                },
                { "$group": { "_id": "$userId" } }
            ],
            "as": "usersAboveAverage",
            "let": { "avgValue": "$avgOrderValue" }
        }
    },

    # 3: Auflösen
    { "$unwind": "$usersAboveAverage" },

    # 4: Nutzerinformationen anhängen
    {
        "$lookup": {
            "from": "users",
            "localField": "usersAboveAverage._id",
            "foreignField": "_id",
            "as": "user"
        }
    },
    { "$unwind": "$user" },

    # 5: Ausgabe formatieren
    {
        "$project": {
            "_id": "$user._id",
            "name": "$user.name",
            "email": "$user.email"
        }
    }
]

result = list(db.orders.aggregate(pipeline))
result

[{'_id': '65f100000000000000000048',
  'name': 'Bob Schneider',
  'email': 'bob.schneider48@example.com'},
 {'_id': '65f100000000000000000008',
  'name': 'Lukas Hoffmann',
  'email': 'lukas.hoffmann8@example.com'},
 {'_id': '65f100000000000000000034',
  'name': 'Alice Becker',
  'email': 'alice.becker34@example.com'},
 {'_id': '65f100000000000000000057',
  'name': 'Eva Hoffmann',
  'email': 'eva.hoffmann57@example.com'},
 {'_id': '65f100000000000000000026',
  'name': 'Uwe Schäfer',
  'email': 'uwe.schaefer26@example.com'},
 {'_id': '65f100000000000000000053',
  'name': 'Zoe Schneider',
  'email': 'zoe.schneider53@example.com'},
 {'_id': '65f100000000000000000027',
  'name': 'Lukas Meyer',
  'email': 'lukas.meyer27@example.com'},
 {'_id': '65f100000000000000000055',
  'name': 'Charlie Becker',
  'email': 'charlie.becker55@example.com'},
 {'_id': '65f100000000000000000028',
  'name': 'Tina Koch',
  'email': 'tina.koch28@example.com'},
 {'_id': '65f100000000000000000054',
  'name': 'Charl

## Task 1 – Count Orders per User
For each user, determine how many orders they have placed. Sort the results in descending order based on the order count.

**Expected output fields:**
- `userId`
- `name`
- `orderCount`

---

In [140]:
pipeline_my = [
    # 1. Group orders by userId
    {
        "$group": {
            "_id": "$userId",
            "orderCount": { "$sum": 1 }
        }
    },

    # 2. Sort by order count descending
    {
        "$sort": { "orderCount": -1 }
    },

    # 3. Join user details
    {
        "$lookup": {
            "from": "users",
            "localField": "_id",
            "foreignField": "_id",
            "as": "user"
        }
    },

    # 4. Unwind joined user
    { "$unwind": "$user" },

    # 5. Project required fields
    {
        "$project": {
            "_id": 0,
            "userId": "$_id",
            "name": "$user.name",
            "orderCount": 1
        }
    }
]

result = list(db.orders.aggregate(pipeline_my))
result

[{'orderCount': 11,
  'userId': '65f100000000000000000057',
  'name': 'Eva Hoffmann'},
 {'orderCount': 10,
  'userId': '65f100000000000000000036',
  'name': 'Ines Schröder'},
 {'orderCount': 10,
  'userId': '65f100000000000000000051',
  'name': 'Ines Wagner'},
 {'orderCount': 9, 'userId': '65f100000000000000000019', 'name': 'Eva Becker'},
 {'orderCount': 8,
  'userId': '65f100000000000000000025',
  'name': 'Alice Hoffmann'},
 {'orderCount': 8,
  'userId': '65f100000000000000000021',
  'name': 'Charlie Schmidt'},
 {'orderCount': 7,
  'userId': '65f100000000000000000008',
  'name': 'Lukas Hoffmann'},
 {'orderCount': 7,
  'userId': '65f100000000000000000042',
  'name': 'Olivia Fischer'},
 {'orderCount': 7,
  'userId': '65f100000000000000000055',
  'name': 'Charlie Becker'},
 {'orderCount': 6,
  'userId': '65f100000000000000000040',
  'name': 'Hannes Wagner'},
 {'orderCount': 6,
  'userId': '65f100000000000000000048',
  'name': 'Bob Schneider'},
 {'orderCount': 6,
  'userId': '65f100000000

## Task 2 – Average Order Value
Compute the average total order value across all orders.
Additionally, include the minimum and maximum order value found.

In [141]:
pipeline_my = [
    {
        "$group": {
            "_id": None,
            "avgOrderValue": { "$avg": "$payment.totalAmount" },
            "minOrderValue": { "$min": "$payment.totalAmount" },
            "maxOrderValue": { "$max": "$payment.totalAmount" }
        }
    }
]

result = list(db.orders.aggregate(pipeline_my))
result

[{'_id': None,
  'avgOrderValue': 868.71068,
  'minOrderValue': 11.53,
  'maxOrderValue': 3691.32}]

## Task 3 – Product Revenue Ranking
For each product, calculate the total revenue generated.

**Definition:**
```
revenue = SUM(quantity × priceAtPurchase)
```

Sort products by total revenue in descending order.

**Expected output fields:**

- `productId`
- `name`
- `totalRevenue`

---

In [142]:
pipeline_my = [
    # 1. Unwind items array
    { "$unwind": "$items" },

    # 2. Group by productId and compute revenue
    {
        "$group": {
            "_id": "$items.productId",
            "totalRevenue": {
                "$sum": {
                    "$multiply": [
                        "$items.quantity",
                        "$items.priceAtPurchase"
                    ]
                }
            }
        }
    },

    # 3. Sort by revenue descending
    { "$sort": { "totalRevenue": -1 } },

    # 4. Join product information
    {
        "$lookup": {
            "from": "products",
            "localField": "_id",
            "foreignField": "_id",
            "as": "product"
        }
    },

    # 5. Unwind joined product
    { "$unwind": "$product" },

    # 6. Final projection
    {
        "$project": {
            "_id": 0,
            "productId": "$_id",
            "name": "$product.name",
            "totalRevenue": 1
        }
    }
]

result = list(db.orders.aggregate(pipeline_my))
result

[{'totalRevenue': 20776.82,
  'productId': '65f200000000000000000027',
  'name': 'Desk Lamp'},
 {'totalRevenue': 18518.41,
  'productId': '65f200000000000000000018',
  'name': 'USB-C Hub'},
 {'totalRevenue': 17331.17,
  'productId': '65f200000000000000000010',
  'name': 'Wireless Mouse'},
 {'totalRevenue': 17324.11,
  'productId': '65f200000000000000000037',
  'name': 'Desk Lamp'},
 {'totalRevenue': 13530.81,
  'productId': '65f200000000000000000032',
  'name': 'Wireless Mouse'},
 {'totalRevenue': 9052.92,
  'productId': '65f200000000000000000033',
  'name': 'Desk Lamp'},
 {'totalRevenue': 9047.66,
  'productId': '65f200000000000000000012',
  'name': 'Noise-Cancelling Headphones'},
 {'totalRevenue': 8858.41,
  'productId': '65f200000000000000000006',
  'name': 'Ergonomic Office Chair'},
 {'totalRevenue': 8825.46,
  'productId': '65f200000000000000000023',
  'name': 'Kitchen Knife Set'},
 {'totalRevenue': 7182.900000000001,
  'productId': '65f200000000000000000034',
  'name': 'Coffee Mu

## Task 4 – User Classification by Spending Behavior
Classify each user based on their *average* order value.

**Categories:**
- `"low"`: < 100 EUR
- `"medium"`: 100–300 EUR
- `"high"`: > 300 EUR

**Expected output fields:**
- `_id`
- `name`
- `avgOrderValue`
- `category`

---

Attention - possible you have to adjust the category boundaries.


In [143]:
pipeline_my = [
    # 1. Group orders per user and compute average order value
    {
        "$group": {
            "_id": "$userId",
            "avgOrderValue": { "$avg": "$payment.totalAmount" }
        }
    },

    # 2. Classify users
    {
        "$addFields": {
            "category": {
                "$switch": {
                    "branches": [
                        {
                            "case": { "$lt": ["$avgOrderValue", 100] },
                            "then": "low"
                        },
                        {
                            "case": {
                                "$and": [
                                    { "$gte": ["$avgOrderValue", 100] },
                                    { "$lte": ["$avgOrderValue", 300] }
                                ]
                            },
                            "then": "medium"
                        },
                        {
                            "case": { "$gt": ["$avgOrderValue", 300] },
                            "then": "high"
                        }
                    ],
                    "default": "medium"
                }
            }
        }
    },

    # 3. Join user information
    {
        "$lookup": {
            "from": "users",
            "localField": "_id",
            "foreignField": "_id",
            "as": "user"
        }
    },
    { "$unwind": "$user" },

    # 4. Final projection
    {
        "$project": {
            "_id": "$user._id",
            "name": "$user.name",
            "avgOrderValue": 1,
            "category": 1
        }
    }
]

result = list(db.orders.aggregate(pipeline_my))
result

[{'avgOrderValue': 809.5166666666665,
  'category': 'high',
  '_id': '65f100000000000000000033',
  'name': 'Olivia Wolf'},
 {'avgOrderValue': 617.634,
  'category': 'high',
  '_id': '65f100000000000000000006',
  'name': 'Zoe Meyer'},
 {'avgOrderValue': 1199.408,
  'category': 'high',
  '_id': '65f100000000000000000030',
  'name': 'Yann Schröder'},
 {'avgOrderValue': 913.1314285714286,
  'category': 'high',
  '_id': '65f100000000000000000055',
  'name': 'Charlie Becker'},
 {'avgOrderValue': 645.9575,
  'category': 'high',
  '_id': '65f100000000000000000058',
  'name': 'Ines Schäfer'},
 {'avgOrderValue': 925.24,
  'category': 'high',
  '_id': '65f100000000000000000001',
  'name': 'Uwe Fischer'},
 {'avgOrderValue': 236.0,
  'category': 'medium',
  '_id': '65f100000000000000000046',
  'name': 'Eva Wagner'},
 {'avgOrderValue': 820.89,
  'category': 'high',
  '_id': '65f100000000000000000029',
  'name': 'Jonas Becker'},
 {'avgOrderValue': 666.5616666666666,
  'category': 'high',
  '_id': '65

## Task 5 – Top 3 Users per Product Category by Spending
For each product category, determine the top 3 users ranked by total spending within that category.

**Rules:**
1. Join orders with products to determine the categories involved.
2. Compute the total spending per user per category
   (`quantity × priceAtPurchase` summed across all orders).
3. For each category, return the top three highest-spending users.

**Expected output structure example:**
```json
{
  "category": "electronics",
  "topUsers": [
    { "userId": "...", "name": "...", "totalSpent": 542.90 },
    { "userId": "...", "name": "...", "totalSpent": 301.50 },
    { "userId": "...", "name": "...", "totalSpent": 199.99 }
  ]
}
```


In [144]:
pipeline_my = [
    # 1. Unwind order items
    { "$unwind": "$items" },

    # 2. Join product to get category
    {
        "$lookup": {
            "from": "products",
            "localField": "items.productId",
            "foreignField": "_id",
            "as": "product"
        }
    },
    { "$unwind": "$product" },

    # 3. Compute spending per item
    {
        "$addFields": {
            "itemSpent": {
                "$multiply": [
                    "$items.quantity",
                    "$items.priceAtPurchase"
                ]
            }
        }
    },

    # 4. Group by category + user
    {
        "$group": {
            "_id": {
                "category": "$product.category",
                "userId": "$userId"
            },
            "totalSpent": { "$sum": "$itemSpent" }
        }
    },

    # 5. Sort users by spending per category
    {
        "$sort": {
            "_id.category": 1,
            "totalSpent": -1
        }
    },

    # 6. Group by category and collect users
    {
        "$group": {
            "_id": "$_id.category",
            "users": {
                "$push": {
                    "userId": "$_id.userId",
                    "totalSpent": "$totalSpent"
                }
            }
        }
    },

    # 7. Slice top 3 users
    {
        "$project": {
            "_id": 0,
            "category": "$_id",
            "topUsers": { "$slice": ["$users", 3] }
        }
    },

    # 8. Join user names
    {
        "$lookup": {
            "from": "users",
            "localField": "topUsers.userId",
            "foreignField": "_id",
            "as": "userInfo"
        }
    },

    # 9. Merge user names into topUsers
    {
        "$addFields": {
            "topUsers": {
                "$map": {
                    "input": "$topUsers",
                    "as": "u",
                    "in": {
                        "userId": "$$u.userId",
                        "totalSpent": "$$u.totalSpent",
                        "name": {
                            "$arrayElemAt": [
                                "$userInfo.name",
                                {
                                    "$indexOfArray": [
                                        "$userInfo._id",
                                        "$$u.userId"
                                    ]
                                }
                            ]
                        }
                    }
                }
            }
        }
    }
]

result = list(db.orders.aggregate(pipeline_my))
result

[{'category': 'kitchen',
  'topUsers': [{'userId': '65f100000000000000000036',
    'totalSpent': 4189.67,
    'name': 'Ines Schröder'},
   {'userId': '65f100000000000000000057',
    'totalSpent': 2046.7599999999998,
    'name': 'Eva Hoffmann'},
   {'userId': '65f100000000000000000005',
    'totalSpent': 2013.45,
    'name': 'Hannes Wolf'}],
  'userInfo': [{'_id': '65f100000000000000000005',
    'name': 'Hannes Wolf',
    'email': 'hannes.wolf5@example.com',
    'createdAt': '2025-01-15T07:16:19',
    'address': {'street': 'Gartenweg 112',
     'city': 'München',
     'zip': '80331',
     'country': 'DE'},
    'tags': ['premium', 'newsletter']},
   {'_id': '65f100000000000000000036',
    'name': 'Ines Schröder',
    'email': 'ines.schroeder36@example.com',
    'createdAt': '2025-01-06T04:48:07',
    'address': {'street': 'Oak Avenue 93',
     'city': 'Frankfurt',
     'zip': '60311',
     'country': 'DE'},
    'tags': ['premium']},
   {'_id': '65f100000000000000000057',
    'name': 'Eva